# Problem Statement

A small engineering consulting firm has 3 senior designers available to work on the firm's 4 current projects over the next 2 weeks. Each designer has 80 hours to split among the projects, and the following table shows the manager's scoring $(0=$ nil to $100=$ perfect $)$ of the capability of each designer to contribute to each project, along with his estimate of the hours that each project will require.


|      Designer     | Project 1    | Project 2    | Project 3    | Project 4    |
|----------|--------------|--------------|--------------|--------------|
| 1        | 90           | 80           | 10           | 50           |
| 2        | 60           | 70           | 50           | 65           |
| 3        | 70           | 40           | 80           | 85           |



|     **Required:**      | Project 1    | Project 2    | Project 3    | Project 4    |
|-----------|--------------|--------------|--------------|--------------|
| **Hours** | 70           | 50           | 85           | 35           |


#### Imports

In [83]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import shutil
import sys
import os.path
from pyomo.environ import *

import pyomo.environ as pe
import pyomo.opt as po

#### Defining Data

In [84]:
Engg = {'E1','E2','E3'}    # Check the type by: type(Engg)

In [85]:
Proj = {'P1','P2','P3','P4'}

In [86]:
score = {
    ('E1','P1'):90,
    ('E1','P2'):80,
    ('E1','P3'):10,
    ('E1','P4'):50,
    ('E2','P1'):60,
    ('E2','P2'):70,
    ('E2','P3'):50,
    ('E2','P4'):65,
    ('E3','P1'):70,
    ('E3','P2'):40,
    ('E3','P3'):80,
    ('E3','P4'):85,
}   # Dictionary with tuples as keys (based on our defined sets)

In [87]:
hours_needed = {
    ('P1'):70,
    ('P2'):50,
    ('P3'):85,
    ('P4'):35,
}

In [88]:
max_hours = 80

## Model

Let the design engineers be set $E$ with $E_{i} \; :i \in [1,2,3]$ and the projects be $P$ with  $P_{j} \; :j \in [1,2,3,4]$. We can model the problem as allocation of the number of hours $H_{ij}$ with each design engineer $E_{i}$ $\forall i $ that are being put onto the projects $P_{j}$ $\forall j $, given the $i^{th}$ engineer $E$ works on $j^{th}$ project with given score $e_{ij}$.

Let the maximum hours available with each engineer be $H_{max}$ and the required number of hours for each project $P_j$ be $R_{j} \; \forall j $. Thus, then the mathematical formulation can be made as:

$$
\text{Maximize } \sum_{i \in E} \sum_{j \in P} H_{ij} e_{ij}
$$

Subject to:

$$
\sum_{j \in P} H_{ij} \leq H_{max} \;\; \forall i \in E
$$

$$
\sum_{i \in E} H_{ij} \geq R_j \;\; \forall j \in P
$$

$$
H_{ij} \geq 0 \;\; \forall i,j
$$


## Implement

In [89]:
m = pe.ConcreteModel()

#### Initializing Sets

In [ ]:
m.Engg = pe.Set(initialize=Engg)
m.Proj = pe.Set(initialize=Proj)     # Ignore if any warnings appear

#### Initializing Parameters

In [91]:
m.score = pe.Param(m.Engg, m.Proj, initialize=score)
m.hours_needed = pe.Param(m.Proj, initialize=hours_needed)
m.max_hours = pe.Param(initialize=max_hours)

#### Initializing [Variables](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html) 
> Note: Refer the documnetation to also note other Predefined Virtual Sets

In [92]:
m.H = pe.Var(m.Engg, m.Proj, domain=pe.NonNegativeReals) 


#### Defining Objective

In [93]:
obj_expr = sum(m.H[i,j]*m.score[i,j] 
               for i in m.Engg for j in m.Proj)
m.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

#### Defining [Constraints](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html)
> Note: Refer the documentation to see other ways to write the constraints, such as using a Constraint List

In [94]:
def maxhour_rule(m,i):
    return sum(m.H[i,j] for j in m.Proj) <= m.max_hours

m.maximum_hours =  pe.Constraint(m.Engg, rule=maxhour_rule)

In [95]:
def projhour_rule(m,j):  
    return sum(m.H[i,j] for i in m.Engg) >= m.hours_needed[j]

m.proj_hours = pe.Constraint(m.Proj, rule=projhour_rule)

## Solve and Postprocess

In [96]:
solver = po.SolverFactory('gurobi')
results = solver.solve(m, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-19
Read LP format model from file /var/folders/ch/190yzmt56zsfp9c_z9g_fcjc0000gn/T/tmpr57wb_41.pyomo.lp
Reading time = 0.00 seconds
x1: 7 rows, 12 columns, 24 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 21.6.0 21G1974)

CPU model: Intel(R) Core(TM) i5-5250U CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x55465dd5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 8e+01]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+32   1.200000e+31   7.500000e+02      0s
       7    1.8825000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations 

In [100]:
print("Optimal Assignment Objective: ",pe.value(m.obj))

Optimal Assignment Objective:  18825.0


In [98]:
for i in m.Engg:
    for j in m.Proj:
        print("Engg",i," working on Project",j,"= ",pe.value(m.H[i,j])," Hours")


Engg E2  working on Project P1 =  0.0  Hours
Engg E2  working on Project P4 =  35.0  Hours
Engg E2  working on Project P2 =  40.0  Hours
Engg E2  working on Project P3 =  5.0  Hours
Engg E1  working on Project P1 =  70.0  Hours
Engg E1  working on Project P4 =  0.0  Hours
Engg E1  working on Project P2 =  10.0  Hours
Engg E1  working on Project P3 =  0.0  Hours
Engg E3  working on Project P1 =  0.0  Hours
Engg E3  working on Project P4 =  0.0  Hours
Engg E3  working on Project P2 =  0.0  Hours
Engg E3  working on Project P3 =  80.0  Hours


In [99]:
for i in m.Engg:
    h = pe.value(sum(m.H[i,j] for j in m.Proj))
    print("Hours Worked by Engineer",i,"= ",h," Hours")

Hours Worked by Engineer E2 =  80.0  Hours
Hours Worked by Engineer E1 =  80.0  Hours
Hours Worked by Engineer E3 =  80.0  Hours
